In [23]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical

In [24]:
def identity_block(x, filter):
    x_input = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (1,1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x) # note that batch normalization should be performed over channels after a convolution,
    # for example if your dimension order are [batch, height, width, channel], you want to use axis=3. Basically you choose the axis index which represents your channels.
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    # Add Residue
    x = tf.keras.layers.Add()([x, x_input])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [19]:
def conv_block(x, filter):
    x_conv = tf.keras.layers.Conv2D(filter, (1,1), padding = 'same')(x)
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (1,1), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x) # note that batch normalization should be performed over channels after a convolution,
    # for example if your dimension order are [batch, height, width, channel], you want to use axis=3. Basically you choose the axis index which represents your channels.
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    # Add Residue
    x = tf.keras.layers.Add()([x, x_conv])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [20]:
def ResNet34():
  block_layers = [3, 4, 6, 3]
  shape = (32, 32, 3)
  n_classes = 10
  filter_size = 64
  # Define the input layer
  x_input = tf.keras.layers.Input(shape) 
  x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input) 
  # Define the first convoultional layer with it's maxpooling
  x = tf.keras.layers.Conv2D(64, (7,7),strides = 2, padding = 'same')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)

  for i in range(len(block_layers)):

      if(i==0):
        # No conv block is required  for the first group
        for j in range(block_layers[i]):
          x = identity_block(x, filter_size)
        else:
          filter_size = filter_size * 2
          x = conv_block(x, filter_size)
          for j in range(block_layers[i]-1):
            x = identity_block(x, filter_size)
  # Develop the final layers
  x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(512, activation = 'relu')(x)
  x = tf.keras.layers.Dense(n_classes, activation = 'softmax')(x)

  model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
  return model



In [ ]:
model = ResNet34()
model.summary()

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [7]:
# Transform label indices to one-hot encoded vectors
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Normalization of pixel values (to [0-1] range)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
sgd = tf.keras.optimizers.legacy.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,y_train, epochs=15, batch_size=32, verbose=2, validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()